In [37]:
from gensim.models import word2vec, Word2Vec
import pandas as pd
from bs4 import BeautifulSoup
import nltk, nltk.data
from nltk.corpus import stopwords
import re
import warnings
warnings.filterwarnings('ignore')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### #1. Обучите модель word2vec. Оцените время обучения модели, используя модуль time.

Перед тем, как обучить модель, разобьем наши тексты на список предложений, где каждое предложение представляет из себя список слов.

Для начала считаем данные.

In [40]:
unlabeledData = pd.read_csv("unlabeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

testData = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

labeledData = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

Необработанный текст выглядит как-то так.

In [41]:
testData.review[0]

'"Naturally in a film who\'s main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce\'s short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."'

Из общей коллекции текстов создадим список из предложений, в котором каждое преждложение представляет из себя список слов, используя функции convert_to_sentences и convert_to_wordlist.

In [38]:
"""Функция преобразует входной документ в последовательность слов."""
def convert_to_wordlist(text):
    # избавляемся от html тегов
    clear_text = BeautifulSoup(text, "lxml").get_text()
    
    # избавляемся от пунктуации и цифр
    clear_text = re.sub("[^a-zA-Z]"," ", clear_text)
    
    # приводим к нижнему регистру и разбиваем
    words = clear_text.lower().split()
    
    return(words)

In [39]:
"""Функция разбивает входной документ на список предложений,
   где каждое предложение представляет из себя список слов."""
def convert_to_sentences(text, tokenizer):
    # Используем NLTK tokenizer, чтобы разбить параграф на предложения.
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        # Пустые предложения мы не рассматриваем
        if len(raw_sentence) > 0:
            # Разбиваем непустое предложение на список слов.
            sentences.append(convert_to_wordlist( raw_sentence))

    return sentences

In [42]:
%%time
df_list = [labeledData, unlabeledData, testData]

sentences = []

for df in df_list:
    for review in df.review:
        sentences += convert_to_sentences(review, tokenizer)

Wall time: 7min 2s


В итоге у нас получилось больше одного миллиона предложений.

In [43]:
len(sentences)

1056938

Каждое предложение в списке спиков представляет из себя совокупность слов.

In [44]:
sentences[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

Обучим нашу модель word2vec, используя данные, которые мы подготовили. Для обучения будем использовать skip-gram, потому что выборка текстов небольшая. Размер окна положим равным 10, субсэмплирование определим на уровне 1e-3.

In [46]:
# %%time

# params = {
#     'sentences' : sentences,
#     'size' : 300,
#     'window' : 10,
#     'sample' : 1e-3,
#     'workers' : 4,
#     'min_count' : 35,
#     'sg' : 1
# }

# обучаем нашу модель word2vec

# model = Word2Vec(**params)

# model.init_sims(replace=True)

Wall time: 15min 30s


Сохраним обученную модель, чтобы не обучать ее каждый раз заново.

In [76]:
# model_name = "model_khorinr"

# сохраняем нашу модель, чтобы не обучать ее заново и использовать в будущем
# model.save(model_name)

Загружаем модель, обученную ранее.

In [4]:
model = Word2Vec.load('model_khorinr')

### #2. Приведите 5-10 примеров использования .most_similar для определения близких слов.

Найдем топ-5 похожих слов, используя функцию __.most_similar__.

In [47]:
model.most_similar('sorrow', topn=5)

[('sadness', 0.7091981768608093),
 ('despair', 0.6270290613174438),
 ('helplessness', 0.6189005374908447),
 ('pain', 0.5961489677429199),
 ('heartache', 0.5872154235839844)]

In [48]:
model.most_similar('intelligent', topn=5)

[('smart', 0.620033323764801),
 ('thoughtful', 0.5956170558929443),
 ('witty', 0.5712857246398926),
 ('clever', 0.5506004095077515),
 ('literate', 0.5464425086975098)]

In [49]:
model.most_similar('iraq', topn=5)

[('afghanistan', 0.612485945224762),
 ('iraqi', 0.5733760595321655),
 ('vietnam', 0.5672841668128967),
 ('war', 0.5660949945449829),
 ('yugoslavia', 0.5499035716056824)]

In [50]:
model.most_similar('football', topn=5)

[('soccer', 0.6276072859764099),
 ('baseball', 0.6067206263542175),
 ('basketball', 0.592818558216095),
 ('rugby', 0.5741385817527771),
 ('coach', 0.560568630695343)]

In [51]:
model.most_similar('pain', topn=5)

[('agony', 0.6327665448188782),
 ('anguish', 0.6102206707000732),
 ('sorrow', 0.5961489677429199),
 ('grief', 0.5655539035797119),
 ('heartache', 0.5548940896987915)]

Модель находит близкие слова достаточно хорошо. В большинстве случаев найденные слова действительно являются синонимами.

### #3. Приведите 5-10 примеров использования most_similar для определения ассоциаций (А к Б, как В к?).

Для нахождения ассоциаций между словами так же будем использовать функцию most_similar.
<p>Попробуем следующие примеры слов:</p>
* big bigger bad
* man son girl
* he his she
* russia russian america
* boy man girl
* go went run

В итоге получаем следующие ассоциации.

In [56]:
words_examples = ['big bigger bad',
                  'man son woman',
                  'he his she',
                  'russia russian america',
                  'boy man girl',
                  'go went run']

for example in words_examples:
    a, b, c = example.split()
    association = model.most_similar([c, b], [a])[0][0]
    print("'%s' к '%s' как '%s' к '%s'\n" % (a, b, c, association))

'big' к 'bigger' как 'bad' к 'worse'

'man' к 'son' как 'woman' к 'daughter'

'he' к 'his' как 'she' к 'her'

'russia' к 'russian' как 'america' к 'american'

'boy' к 'man' как 'girl' к 'woman'

'go' к 'went' как 'run' к 'ran'



Видно, что все ассоциации определены верно.

### #4. Приведите 5-10 примеров использования .doesnt_match для определения лишнего слова.

Попытаемся найти лишние слова, используя функцию .doesnt_match для следующих примеров.

In [57]:
model.doesnt_match("shotgun clutch pistol bullet projectile".split())

'clutch'

In [58]:
model.doesnt_match("poet judge playwright dramatist".split())

'judge'

In [59]:
model.doesnt_match("smart intelligent silly clever".split())

'silly'

In [60]:
model.doesnt_match("blue yellow attack green red".split())

'attack'

In [61]:
model.doesnt_match("refund raise odd procure repayment".split())

'odd'

Как видно из примеров, модель определяет лишние слова достаточно хорошо. Из примеров все лишние слова найдены корректно.

### #5.1. Попробуйте найти такие пары и тройки слов, для которых НЕ выполняются свойства коммутативности и транзитивности относительно операции определения близких слов (входить в топ-3 по .most_similar).

### Коммутативность

1) Пары слов __experiences__ и __life__ не являются коммутативными. Так, experiences входит в top-3 по близости к слову life, но life в топ-3 по близости к слову experiences не входит.

In [62]:
model.most_similar('life', topn=3)

[('lives', 0.5257035493850708),
 ('sorrows', 0.4022024869918823),
 ('experiences', 0.38946056365966797)]

In [63]:
model.most_similar('experiences', topn=3)

[('experience', 0.5324373841285706),
 ('joys', 0.4490772485733032),
 ('discoveries', 0.440945029258728)]

2) Пары слов __bobbing__ и __water__ так же не являются коммутативными.

In [64]:
model.most_similar('water', topn=3)

[('swimming', 0.5230377912521362),
 ('fish', 0.49989980459213257),
 ('bobbing', 0.4936462342739105)]

In [67]:
model.most_similar('bobbing', topn=3)

[('sails', 0.638393759727478),
 ('flapping', 0.6207567453384399),
 ('swirling', 0.6143648028373718)]

### Транзитивность

1) Тройка слов __god, jesus, christ__ не является транзитивной.
<p>God входит в топ-3 по most_similar слова jesus, jesus входит в топ-3 по most_similar слова christ, но god __не__ входит в топ-3 по most_similar слова christ.</p>

In [68]:
model.most_similar('jesus', topn=3)

[('christ', 0.7511848211288452),
 ('god', 0.49731725454330444),
 ('judas', 0.491919606924057)]

In [69]:
model.most_similar('christ', topn=3)

[('jesus', 0.7511848211288452),
 ('teachings', 0.4705061614513397),
 ('judas', 0.46374014019966125)]

2) Тройка слов __russia, ussr, germany__ так же не является транзитивной.
<p>USSR входит в топ-3 по most_similar слова russia, russia входит в топ-3 по most_similar слова germany, но USSR __не__ входит в топ-3 по most_similar слова germany.</p>

In [70]:
model.most_similar('russia', topn=3)

[('germany', 0.6032990217208862),
 ('ussr', 0.5540001392364502),
 ('bulgaria', 0.5516657829284668)]

In [71]:
model.most_similar('germany', topn=3)

[('europe', 0.6586641073226929),
 ('poland', 0.6117411851882935),
 ('russia', 0.6032990217208862)]

### #5.2. Попробуйте найти такие пары и тройки слов, для которых выполняются свойства коммутативности и транзитивности относительно операции определения близких слов (входить в топ-3 по .most_similar).

### Коммутативность

1) Пары слов __iraq__ и __afghanistan__ являются коммутативными. Они оба входят в топ-3 по близости относительно друг друга.

In [72]:
model.most_similar('iraq', topn=3)

[('afghanistan', 0.612485945224762),
 ('iraqi', 0.5733760595321655),
 ('vietnam', 0.5672841668128967)]

In [73]:
model.most_similar('afghanistan', topn=3)

[('taliban', 0.682716965675354),
 ('iraq', 0.612485945224762),
 ('pakistan', 0.6068713665008545)]

2) Пары слов __jesus__ и __christ__ так же являются коммутативными.

In [74]:
model.most_similar('jesus', topn=3)

[('christ', 0.7511848211288452),
 ('god', 0.49731725454330444),
 ('judas', 0.491919606924057)]

In [75]:
model.most_similar('christ', topn=3)

[('jesus', 0.7511848211288452),
 ('teachings', 0.4705061614513397),
 ('judas', 0.46374014019966125)]

### Транзитивность

1) Тройка слов __launcher, handgun, grenade__ является транзитивной.
<p>Launcher входит в топ-3 по most_similar слова handgun, handgun входит в топ-3 по most_similar слова grenade, launcher так же входит в топ-3 по most_similar слова grenade.</p>

In [77]:
model.most_similar('handgun', topn=3)

[('dart', 0.6749066114425659),
 ('launcher', 0.6716697812080383),
 ('pistol', 0.6707010269165039)]

In [78]:
model.most_similar('grenade', topn=3)

[('launcher', 0.723590612411499),
 ('grenades', 0.6451822519302368),
 ('handgun', 0.6407240033149719)]

2) Тройка слов __portugal__, __spain__, __france__ так же является транзитивной.
<p>Portugal входит в топ-3 по most_similar слова spain, spain входит в топ-3 по most_similar слова france, portugal так же входит в топ-3 по most_similar слова france.</p>

In [100]:
model.most_similar('spain', topn=3)

[('france', 0.6373680830001831),
 ('portugal', 0.6348042488098145),
 ('italy', 0.5802099704742432)]

In [101]:
model.most_similar('france', topn=3)

[('belgium', 0.6461061239242554),
 ('spain', 0.6373680830001831),
 ('portugal', 0.6353395581245422)]

После выполнения всех пунктов данного дз можно сказать, что Word2Vec является удобным и мощным инструментов для выявления семантической близости слов, расчета векторных представлений слов, а также для нахождения ассоциаций и выявления лишних слов в предложениях. Умело обученная модель успешно способна справиться со всеми задачами такого типа.